In [589]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import os
from sklearn.metrics import mean_squared_error

In [590]:
channels = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7'
            , 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz'
            , 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2','AUX_1', 'ACC_X', 'ACC_Y', 'ACC_Z']
removed_channels = ['Fp1', 'F8', 'F7', 'Fp2', 'F3', 'F4']
# TODO ADD AUX WHEN CONNECTED
eye_data_names = ['x', 'y', 'z', 'left_blink', 'right_blink', 'both_blink', 'left_openness', 'right_openness']

columns =  eye_data_names + channels

data, header = pyxdf.load_xdf('data/LSLDataV2/LSLDataLeftFourTimes2/eeg-and-eyetracking.xdf')


In [591]:
def get_time_stamps(stamps: list):
    start = []
    end = []
    more_than_one = False

    for i in range(len(stamps)):
        if i == (len(stamps)-1):
            if(stamps[i-1] == (stamps[i] - 1)):
                end.append(stamps[i])
            break

        if(stamps[i] == (stamps[i+1] - 1)):
            if not more_than_one:
                start.append(stamps[i])
            more_than_one = True
            continue
        else:
            if more_than_one:
                end.append(stamps[i])
                more_than_one = False

    return start, end

In [592]:
def plot_eeg_blink_single_norm(df,df_ts, start, end,column,rang = 50, scale=1, flip = False):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    axis.plot(df_ts['time'][start-rang:end+rang], (df_norm + plus) *(-1 if flip else 1) *scale, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Amplitude ")
    axis.set_title(column)
    plt.legend()
    plt.show()

def plot_eeg_blink_single(df,df_ts, start, end,column,rang = 50):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")

    axis.plot(df_ts['time'][start-rang:end+rang], df[column], label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], df['left_openness'], color='orange', label = "Eye openness left - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], df['right_openness'], color='violet', label = "Eye openness right - SRanipal")
    axis.set_title(column)
    plt.legend()
    plt.show()

In [593]:

def init_data(data, shift = 0):
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = columns
        df_stream['AUX_1'] = df_stream['AUX_1']+ 1000000
        if shift != 0:
            df_stream['AUX_1'] = df_stream['AUX_1'].shift(shift)
        return stream, df_stream, df_stream_ts

stream, df_stream, df_stream_ts = init_data(data)
def get_blinks_in_df(df):
    left_blinks = df.index[(df['left_blink'] >= 1) & (df['right_blink'] != 1)].tolist()
    right_blinks = df.index[(df['right_blink'] >= 1) & (df['left_blink'] != 1)].tolist()
    both_blink = df.index[df['both_blink'] == 1].tolist()
    return left_blinks,right_blinks, both_blink

left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)

In [594]:
avg_time = np.diff(df_stream_ts.to_numpy().flatten()).tolist()

In [595]:
start, end = get_time_stamps(left_blinks)
flip = False
chosen_channel = 'AUX_1'
print(start)
print(end)


[1201, 1510, 1858]
[1278, 1576, 1917]


In [596]:
from scipy.signal import butter, lfilter
# Sample rate and desired cutoff frequencies (in Hz).
fs = 500
lowcut = 1
highcut = 28



def butter_bandpass(lowcut, highcut, fs, order=4):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def plot_eeg_blink_single_norm_butter(df,df_ts, start, end,column,rang = 50, scale=1, flip = False, order = 4):
    plt.figure(figsize=(10,6))
    df = df[start - rang:end + rang]
    df.columns = columns
    axis = plt.subplot()
    axis.axvline(df_ts['time'][start], color='green', label = "Blink start - TobiiXR")  # y = 0
    axis.axvline(df_ts['time'][end], color='red', label = "Blink End - TobiiXR")
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]

    y = butter_bandpass_filter((df_norm + plus) *(-1 if flip else 1) *scale, lowcut, highcut, fs, order=order)
    peaks, _ = signal.find_peaks(y[rang-7:rang+30])
    peaks = [peak + rang-7 for peak in peaks]

    axis.plot(df_ts['time'][start-rang:end+rang], y, label = "EMG")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['left_openness'] / df['left_openness'].abs().max())
               *(-1)+ 1, color='orange', label = "Eye openness - SRanipal")
    axis.plot(df_ts['time'][start-rang:end+rang], (df['right_openness'] / df['right_openness'].abs().max())
               *(-1)+ 1, color='violet', label = "Eye openness right - SRanipal")
    axis.set_xlabel("Time normalized (seconds)")
    axis.set_ylabel("Normalized Amplitude")
    for i in peaks:
        plt.plot(df_ts['time'][i+ start - rang], y[i], marker="o", markersize=5, markeredgecolor="turquoise", markerfacecolor="black", label="Local maximum " + str(y[i].round(2)) + ", " + str(i+ start - rang))
    axis.set_title(column)
    plt.legend()
    plt.show()

def get_peaks(df, start, end, column, rang=50, scale=1, order=4, flip=False):
    df = df[start - rang:end + rang]
    df_norm = (df[column].abs() / df[column].abs().max())
    plus = -df_norm[start - rang]
    y = butter_bandpass_filter((df_norm + plus) *(-1 if flip else 1) *scale, lowcut, highcut, fs, order=order)
    peaks, _ = signal.find_peaks(y[rang-20:rang+30])
    peaks = [peak + rang-20 for peak in peaks]
    if len(peaks) == 0:
        #print(df)
        return 0
    peak = peaks[np.argmax([y[i] for i in peaks])]+ start - rang
    return  peak if end + rang > peak > start - rang else start



In [597]:


def find_shift(start, peaks):
    differences_start = []
    differences_end = []
    for i in range(len(peaks)):
        differences_start.append(start[i]-peaks[i])
    return differences_start

def plot_table(df):
    #make this example reproducible

    #define figure and axes
    fig, ax = plt.subplots()
    #hide the axes
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')

    #create data

    #create table
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    table.scale(2.5, 2.5)
    table.set_fontsize(30)
    plt.show()

In [598]:

def run_files(paths, path='data/LSLDataV3/', both = True, plot = False, shift_init=0):
    peak_info = []
    avg_time = []
    for (index,y) in enumerate(paths):
        data, header = pyxdf.load_xdf(path+str(y)+'/dejittered-fir-eeg-and-dejittered-eyetracking.xdf')
        stream, df_stream, df_stream_ts = init_data(data,shift_init)
        left_blinks, right_blinks, both_blink = get_blinks_in_df(df_stream)
        start, end = get_time_stamps(both_blink if both else left_blinks)
        avg_time.append(np.diff(df_stream_ts.to_numpy().flatten()).tolist())
        peaks = []
        for i in range(len(start)):
            if i == 9:
                continue
            if "LeftTenTimes6" in y or "LeftTenTimes7" in y:
                peaks.append(get_peaks(df_stream, start[i], end[i], chosen_channel, rang=50, scale=5, flip=True, order=4))
            else:
                peaks.append(get_peaks(df_stream, start[i], end[i], chosen_channel, rang=50, scale=5, flip=both, order=4))

        shift = find_shift(start, peaks)
        peak_info.append(shift)

    n = len(max(peak_info, key=len))
    t = [x + [None]*(n-len(x)) for x in peak_info]
    numpy_array_peak_info = np.array(t)
    df_peak = pd.DataFrame(numpy_array_peak_info)
    df_peak = df_peak.set_index([paths])
    df_peak['time between index (s)'] = [i[0] for i in avg_time]
    df_peak['Sum (indexes)'] = df_peak.sum(axis=1)
    df_peak['Avg (indexes)'] = round(df_peak.mean(axis=1),3)
    df_peak['Avg time (s)'] = round(df_peak['Avg (indexes)']*df_peak['time between index (s)'],5)
    if plot:
        plot_table(df_peak)
    return df_peak



In [599]:
from IPython.core.display_functions import display

paths = [name for name in os.listdir("./data/LSLDataV3")]
both = [name for name in paths if "Both" in name]
left = [name for name in paths if "Left" in name]


df_both = run_files(both, both=True , plot=False)
#df_both = df_both.drop(df_both.columns[[1]], inplace=True, axis=1)
display(df_both)
df_left = run_files(left, both=False , plot=False)
#df_left = df_left.drop(df_left.columns[[1]], inplace=True, axis=1)
display(df_left)


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataBothTenTimes2,-13,-15,-14,-18,-13,-12,-11,-14,-10,0.008,-119.992,-21.817,-0.17454
LSLDataBothTenTimes5,-13,-13,-13,-15,-13,-14,-13,-13,-13,0.008,-119.992,-21.817,-0.17454
LSLDataBothTenTimes4,-12,-12,-14,-15,-12,-15,-15,-14,-15,0.008,-123.992,-22.544,-0.18035
LSLDataBothTenTimes1,-13,-12,-18,-15,-12,-14,-13,-11,-13,0.008,-120.992,-21.999,-0.17599
LSLDataBothTenTimes6,-11,-8,-11,-11,-13,-12,-11,-12,-13,0.008,-101.992,-18.544,-0.14835
LSLDataBothTenTimes7,-12,-11,-11,-11,-14,-11,-13,-12,-14,0.008,-108.992,-19.817,-0.15854


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeftTenTimes1,-15,-14,-16,-14,-16,-14,-14,-10,-15,0.008,-127.992,-23.271,-0.18617
LSLDataLeftTenTimes7,-12,-11,-13,-12,-12,-8,-11,-5,-6,0.008,-89.992,-16.362,-0.13090
LSLDataLeftTenTimes3,-19,-18,-16,-17,-20,-16,-16,-17,-18,0.008,-156.992,-28.544,-0.22835
LSLDataLeftTenTimes4,-18,-20,-16,-17,-16,-22,-13,-21,-19,0.008,-161.992,-29.453,-0.23562
LSLDataLeftTenTimes6,-11,-8,-10,-9,-8,-11,-15,-7,-14,0.008,-92.992,-16.908,-0.13526
LSLDataLeftTenTimes2,-19,-17,-19,-19,-17,-15,-18,-16,-14,0.008,-153.992,-27.999,-0.22399


In [600]:
def get_offset(df):
    avg_offset = df['Avg time (s)'].sum()/df.shape[0]
    avg_offset_index = df['Avg (indexes)'].sum()/df.shape[0]
    #print(avg_offset_index)
    return avg_offset, avg_offset_index


In [601]:
print("(Seconds, Number of indexes to shift) ---- Both")
print(get_offset(df_both))
print("(Seconds, Number of indexes to shift) ---- Left")
print(get_offset(df_left))

df_both_shift = run_files(both, both=True , plot=False, shift_init=round(get_offset(df_both)[1]))
df_both_shift.loc['Average']= df_both_shift.sum(numeric_only=True, axis=0)/df_both_shift.shape[0]
display(df_both_shift)
df_left_shift = run_files(left, both=False , plot=False, shift_init=round(get_offset(df_left)[1]))
df_left_shift.loc['Average']= df_left_shift.sum(numeric_only=True, axis=0)/df_left_shift.shape[0]
display(df_left_shift)

print(get_offset(df_both_shift))
print(get_offset(df_left_shift))

(Seconds, Number of indexes to shift) ---- Both
(-0.16871833333333333, -21.089666666666663)
(Seconds, Number of indexes to shift) ---- Left
(-0.1900483333333333, -23.75616666666667)


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataBothTenTimes2,8.000000,6.000000,7.000000,3.000000,8.000000,9.0,10.000000,7.000000,11.0,0.008,69.008000,12.5470,0.100380
LSLDataBothTenTimes5,8.000000,8.000000,8.000000,6.000000,8.000000,7.0,8.000000,8.000000,8.0,0.008,69.008000,12.5470,0.100380
LSLDataBothTenTimes4,9.000000,9.000000,7.000000,6.000000,9.000000,6.0,6.000000,7.000000,6.0,0.008,65.008000,11.8200,0.094560
LSLDataBothTenTimes1,8.000000,9.000000,3.000000,6.000000,9.000000,7.0,8.000000,10.000000,8.0,0.008,68.008000,12.3650,0.098920
LSLDataBothTenTimes6,10.000000,13.000000,11.000000,10.000000,8.000000,9.0,10.000000,9.000000,8.0,0.008,88.008000,16.0010,0.128010
LSLDataBothTenTimes7,9.000000,10.000000,10.000000,10.000000,7.000000,10.0,8.000000,9.000000,7.0,0.008,80.008000,14.5470,0.116380
Average,8.666667,9.166667,7.666667,6.833333,8.166667,8.0,8.333333,8.333333,8.0,0.008,73.174667,13.3045,0.106438


,0,1,2,3,4,5,6,7,8,time between index (s),Sum (indexes),Avg (indexes),Avg time (s)
LSLDataLeftTenTimes1,9.000000,10.000000,8.0,10.000000,8.000000,10.0,11.000000,14.000000,9.000000,0.008,89.008,16.183000,0.12946
LSLDataLeftTenTimes7,12.000000,13.000000,11.0,12.000000,12.000000,16.0,13.000000,19.000000,18.000000,0.008,126.008,22.911000,0.18329
LSLDataLeftTenTimes3,5.000000,6.000000,8.0,7.000000,4.000000,9.0,8.000000,7.000000,6.000000,0.008,60.008,10.911000,0.08729
LSLDataLeftTenTimes4,6.000000,4.000000,8.0,7.000000,9.000000,3.0,11.000000,3.000000,5.000000,0.008,56.008,10.183000,0.08146
LSLDataLeftTenTimes6,13.000000,16.000000,-17.0,15.000000,16.000000,13.0,9.000000,17.000000,10.000000,0.008,92.008,16.729000,0.13383
LSLDataLeftTenTimes2,5.000000,7.000000,6.0,5.000000,7.000000,9.0,6.000000,8.000000,10.000000,0.008,63.008,11.456000,0.09165
Average,8.333333,9.333333,4.0,9.333333,9.333333,10.0,9.666667,11.333333,9.666667,0.008,81.008,14.728833,0.11783


(0.10643833333333334, 13.3045)
(0.11782999999999999, 14.728833333333332)


In [602]:
import statistics
from scipy.stats import sem

standard_de_both = statistics.stdev(df_both['Avg time (s)'])
se_both = sem(df_both['Avg time (s)'])
standard_de_left = statistics.stdev(df_left['Avg time (s)'])
se_left = sem(df_left['Avg time (s)'])

print("Standard devation")
print("Both: " + str(standard_de_both))
print("Left: " + str(standard_de_left))

print("-------- SE ----------")
print("Both: " + str(se_both))
print("Left: " + str(se_left))

Standard devation
Both: 0.012445271257255372
Left: 0.047352211106416846
-------- SE ----------
Both: 0.0050807607151335575
Left: 0.019331459233878627


In [603]:
standard_de_both_shift = statistics.stdev(df_both_shift['Avg time (s)'])
se_both_shift = sem(df_both_shift['Avg time (s)'])
standard_de_left_shift = statistics.stdev(df_left_shift['Avg time (s)'])
se_left_shift = sem(df_left_shift['Avg time (s)'])

print("Standard devation - After shift")
print("Both: " + str(standard_de_both_shift))
print("Left: " + str(standard_de_left_shift))

print("-------- SE - After shift ----------")
print("Both: " + str(se_both_shift))
print("Left: " + str(se_left_shift))

Standard devation - After shift
Both: 0.011798308376863
Left: 0.035630346148566486
-------- SE - After shift ----------
Both: 0.004459341408061375
Left: 0.01346700500517928
